Assignment 1: write a program to rotate an image by a given angle (in radians) using bilinear interpolation

Start:
* load image
* set rotation angle 

In [ ]:
import cv2
import numpy as np

# Load the image
img = cv2.imread("/Users/kimjunho/Desktop/컴퓨터비전3-1/assignment1/img_example.JPG")

# Set the rotation angle
ang = np.pi/3

Problem 1: [code by yourself]

Define a function `rotate_img()`, with the following input parameters and return variables:
* img (input): input image as a 3D numpy array of size (H,W,3)
* ang (input): rotation angle in radian
* img_out (output): output image as a 3D numpy array of size (H_out,W_out,3)

This function should include the following processes:
* Define a 2x2 rotation matrix as a numpy array for the given rotation angle 
* Compute output image size and the offset for the output image coordinates by computing the image corner coordinates after rotating the image
* Apply bilinear interpolation when determining the pixel values for the output image 



In [ ]:
def rotate_img(img, ang):
  # compute rotation matrix
  m60 = np.float32([[np.cos(ang), -1*np.sin(ang)],
                        [np.sin(ang), np.cos(ang)]])
  
  # compute output coordinates for image corners
  row, col = img.shape[:2] # 1124 843
  coordinate = np.array([[0,0], [0,row-1], [col-1, 0], [col-1, row-1]]) #(x,y)
  newcoordinate  = []
  for i in range(4):
      newcoordinate.append((m60@coordinate[i].T).T)
  newcoordinate = np.array(newcoordinate)
  print(newcoordinate)
  
  # compute output image size and offset
  col_min, row_min = np.min(newcoordinate, axis=0).astype(int)
  col_max, row_max = np.max(newcoordinate, axis=0).astype(int)
  col_out, row_out = col_max - col_min + 1, row_max - row_min + 1
  print('output(row, col) =', row_out, col_out)
  col_offset, row_offset = -col_min, -row_min
  print('offsets(x,y) =', col_offset, row_offset)
  
  # define output image object
  img_out = np.zeros((row_out, col_out, 3), dtype=img.dtype)
  
  # for each output image coordinate, 
  # compute corresponding original image coordinate
  # get pixel value from original image with interpolation
  for r in range(row_out):
        for c in range(col_out):
            x_in, y_in = np.array([c - col_offset, r - row_offset])@m60
            if x_in >= 0 and y_in >= 0 and x_in <= col - 1 and y_in <= row - 1:
                x_in0, y_in0 = int(np.floor(x_in)), int(np.floor(y_in))
                x_in1, y_in1 = x_in0+1, y_in0+1
                alpha, beta = x_in - x_in0, y_in - y_in0
                img_out[r, c] = (1-beta)*(1-alpha)*img[y_in0,x_in0] + (1-beta)*alpha*img[y_in0,x_in1] + beta*(1-alpha)*img[y_in1,x_in0] + beta*alpha*img[y_in1,x_in1]

  return img_out

Show the output:
* use matplotlib

In [ ]:
import matplotlib.pyplot as plt
img_out = rotate_img(img, ang)
# Convert the image from BGR to RGB format
img_out = cv2.cvtColor(img_out, cv2.COLOR_BGR2RGB)
# Display the image using Matplotlib
plt.imshow(img_out)
plt.show()